In [2]:
print("hello world")

hello world


In [15]:
import numpy as np
import matplotlib as plt

import pycuda.driver as cuda_driver
import pycuda.compiler as cuda_compiler

from pycuda.gpuarray import GPUArray

import IPythonMagic
from Timer import Timer

In [16]:
%setup_logging

Global logger already initialized!


In [17]:
%cuda_context_handler context

Registering context in user workspace
Context already registered! Ignoring


In [32]:
kernel_src = """

__global__ void shmemReduction(float* output, float* input, int size) {
    // First we stride through global memory and compute the maximum for every thread.
    int gid = blockIdx.x * blockDim.x + threadIdx.x;   // blockIdx.x is always zero because we use just one block
    
    float max_value = -9999999.99;   // FIX ME
    for (int i = threadIdx.x; i < size; i = i + blockDim.x) {
        max_value = fmaxf(max_value, input[i]);
    }
    
    // Store the pre-thread maximum in shared memory.
    __shared__ float max_shared[64];
    max_shared[threadIdx.x] = max_value;
    
    // Synchronize so that all threads see the same shared memory.
    __syncthreads();
    
    // Find the maximum in shared memory.
    // Reduce from 64 to 32 elements
    if (threadIdx.x < 32) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 32]);
    }
    __syncthreads();
    // Reduce from 32 to 16 elements
    if (threadIdx.x < 16) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 16]);
    }
    __syncthreads();
    // Reduce from 16 to 8 elements
    if (threadIdx.x < 8) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 8]);
    }
    __syncthreads();
    // Reduce from 8 to 4 elements
    if (threadIdx.x < 4) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 4]);
    }
    __syncthreads();
    // Reduce from 4 to 2 elements
    if (threadIdx.x < 2) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 2]);
    }
    __syncthreads();
    // Reduce from 2 to 1 element
    if (threadIdx.x < 1) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 1]);
    }
    __syncthreads();
    
    // Finally write out to output.
    if (threadIdx.x == 0) {
        output[0] = max_shared[0];
    }
}
"""

kernel_module = cuda_compiler.SourceModule(kernel_src)
kernel_function = kernel_module.get_function("shmemReduction")

/home/ubuntu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu(6): warning: variable "gid" was declared but never referenced




In [53]:
n = 64
a = np.random.random((1, n)).astype(np.float32)

a_g = GPUArray(a.shape, a.dtype)
a_g.set(a)

num_threads = 64
b = np.empty((1, num_threads), dtype=np.float32)
b_g = GPUArray(b.shape, b.dtype)

In [54]:
block_size = (num_threads, 1, 1)
grid_size = (1, 1, 1)

kernel_function(b_g, a_g, np.int32(n), grid=grid_size, block=block_size)

b_g.get(b)

print(a)
print(b)
print(np.max(a))

[[0.8652245  0.6028568  0.49051866 0.37550572 0.30786648 0.21346712
  0.24728757 0.8001967  0.7523666  0.18181866 0.93468153 0.77047676
  0.74495023 0.9404862  0.00633006 0.29288366 0.36989376 0.95670927
  0.2265881  0.50838155 0.5947812  0.38475344 0.1556996  0.82997346
  0.9160598  0.99135363 0.69199544 0.4045     0.8296572  0.5686859
  0.16014126 0.8327087  0.5546283  0.17468268 0.7989543  0.7365818
  0.85785705 0.8810847  0.8704667  0.9980439  0.8701734  0.00577471
  0.7038801  0.6449104  0.15989287 0.81345356 0.5683608  0.57394755
  0.90552723 0.36622635 0.1664252  0.18958624 0.986221   0.9413474
  0.14121401 0.74505246 0.82000947 0.13585423 0.2565578  0.01533947
  0.929867   0.86964154 0.9166347  0.39545536]]
[[0.9980439  0.9684571  0.33426452 0.19167586 0.12459923 0.5402504
  0.34930632 0.2662844  0.14671509 0.78913593 0.71126646 0.23983198
  0.3289858  0.8386073  0.22462337 0.52970004 0.22329433 0.13470535
  0.87593037 0.12188934 0.70764905 0.81207794 0.75761294 0.41417593
  0.